### Theoretical part
1. hoe werkt markdown en doe ik linebreaks
2.

### Experimental part

##### Step 1

In [2]:
import math, random, itertools, numpy as np

In [3]:
def pair_generator():
    """
    A generator that returns pairs of all possible combinations 
    of [N, R, HR] of length 5, with repeated elements.
    """
    pairs = itertools.product(itertools.product(['N', 'R', 'HR'], repeat=5), repeat=2)
    for p in pairs:
        yield p

print('Number of pairs:', len(list(pair_generator())))

print('first 10 pairs:')
pair_gen = pair_generator()
for _ in range(10):
    print(next(pair_gen))

def random_sample(length):
    '''
    Returns a sample pair that
    consists of a production and 
    an experiment list, with as possible
    values {N, R, HR}.
    '''
    values = ['N', 'R', 'HR']
    
    p = [values[random.randint(0, 2)] for _ in range(length)]
    e = [values[random.randint(0, 2)] for _ in range(length)]
    
    return p, e


Number of pairs: 59049
first 10 pairs:
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'N', 'N'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'N', 'R'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'N', 'HR'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'R', 'N'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'R', 'R'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'R', 'HR'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'HR', 'N'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'HR', 'R'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'N', 'HR', 'HR'))
(('N', 'N', 'N', 'N', 'N'), ('N', 'N', 'R', 'N', 'N'))


In [24]:
# Implement Evaluation Measures

import numpy as np

def binary_precision(p, f=None):
    return np.array([0 if x == 'N' else 1 for x in p]).sum() / len(p)

def ndcg(p, relevance_map):
    dcg = np.array([
        (2 ** relevance_map[x] - 1) / np.log2(r + 1) for r, x in enumerate(p, start=1) 
    ])

    return (dcg / dcg.max()).sum() if dcg.max() != 0 else 0

def err(p, relevance_map):
    P = 1
    E = 0
    for r, v in enumerate(p, start=1):
        R = (2 ** relevance_map[v] - 1) / (2 ** max(relevance_map.values()))
        E += P * R / r
        P *= (1-R)
    return E
    
p, e = random_sample(5)

relevance_map = {
    'N': 0,
    'R': 1,
    'HR': 2
}

print('Precision of production algorithm:\t', binary_precision(p))
print('Precision of experimental algorithm:\t', binary_precision(e))

print('EER of production algorithm:\t\t', err(p, relevance_map))
print('EER of experimental algorithm:\t\t', err(e, relevance_map))

print('NDCG of production algorithm:\t\t', ndcg(p, relevance_map))
print('NDCG of experimental algorithm:\t\t', ndcg(e, relevance_map))

Precision of production algorithm:	 0.6
Precision of experimental algorithm:	 0.6
EER of production algorithm:		 0.4239583333333333
EER of experimental algorithm:		 0.484375
NDCG of production algorithm:		 1.87730760955
NDCG of experimental algorithm:		 2.08728650238


In [25]:
# Calculate the 𝛥measure

def avg_delta(n, eval_function, relevance_map):
    return sum(
        [eval_function(x[0], relevance_map) - eval_function(x[1], relevance_map) 
         for x in [random_sample(5) for _ in range(n)]]
    ) 

    
print(avg_delta(5000, binary_precision, relevance_map))
print(avg_delta(5000, err, relevance_map))
print(avg_delta(5000, ndcg, relevance_map))

-10.8
-4.669335937500002
-16.4139716625


In [26]:
def teamdraft_interleaving(docs1, docs2):
    team_a = set()
    team_b = set()
    i = []
    
    while len(set(docs1) - set(i)) > 0 and len(set(docs2) - set(i)):
        if len(team_a) < len(team_b) or (len(team_a) == len(team_b) and random.random() > 0.5):
            k = [x for x in docs1 if not x in i][0]
            i.append(k)
            team_a.add(k)
        else:
            k = [x for x in docs2 if not x in i][0]
            i.append(k)
            team_b.add(k)
    return i, team_a, team_b

docs1, docs2 = ['a', 'b', 'c', 'd'], ['b', 'c', 'd', 'a' ]
p, e = random_sample(5)
p_n = [x for x in range(len(p))]
e_n = [x + len(p) for x in range(len(p))]

i, a, b = teamdraft_interleaving(p_n, e_n)
labels_i = [(p+e)[vis] for vis in i]

print(a, b)
print(p_n, e_n)
print(i)

{0, 1, 2, 3} {5, 6, 7, 8, 9}
[0, 1, 2, 3, 4] [5, 6, 7, 8, 9]
[5, 0, 1, 6, 7, 2, 8, 3, 9]


In [35]:
random.random()

0.576893042361473

In [41]:
from collections import defaultdict

class PBM(object):
    def __init__(self, ranking_size):
        self.alpha = defaultdict(lambda: 1)
        self.gamma = [random.random() for _ in range(ranking_size)]
        
    def estimate(self, S):
        self.gamma = S.sum(axis=0) / len(S)
    
    def predict(self, ranking, query):
        return [g * self.alpha[(u, query)] for g, u in zip(self.gamma, ranking)]
           
    def simulate():
        return something

class RCM:
    def _init_(self):
        self.rho = 0
        
    def estimate(self, clicks):
        self.rho = sum(sum(s) for s in S) / sum(len(s) for s in S)
        
    def predict(self, ranking):
        probs = [self.rho] * len(ranking)
        return probs
            
    def simulate(self, ranking):
        probs = self.predict(ranking)
        for i, (page, prob) in enumerate(zip(ranking, probs)):
            if np.random.random() < prob:
                return i
        return None

S = np.array(
    [[1, 0, 0, 0, 0], 
     [0, 1, 0, 0, 0], 
     [0, 0, 1, 0, 0], 
     [0, 0, 0, 1, 0], 
     [0, 0, 0, 0, 1], 
     [1, 0, 0, 0, 0]]
)

model = PBM(5)
model.estimate(S)

model = RCM()
model.estimate(S)
